In [2]:
import cv2
import numpy as np

# 加载图像
image_path = './crop_map.png'
image = cv2.imread(image_path)

# 定义颜色范围 (使用 HSV 格式)
color_ranges = {
    'F': ((92, 46, 87), (112, 66, 107)),
    '0': ((94, 51, 53), (114, 71, 73)), 
    '1': ((93, 63, 80), (113, 83, 100)),  
    '2': ((79, 65, 83), (99, 85, 103)),   
    '3': ((120, 68, 89), (140, 88, 109)),
    '4': ((104, 63, 86), (124, 83, 106)),
    '5': ((60, 92, 86), (80, 112, 106)),
    '6': ((89, 73, 84), (109, 93, 104)),
}

# 初始化矩阵
num_rows, num_cols = 16, 30
matrix = [[('H', (0, 0, 0)) for _ in range(num_cols)] for _ in range(num_rows)]  # 每个单元格存储 ('H', (H, S, V))

# 将图像转换为 HSV 颜色空间
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 获取图像的尺寸
height, width = image.shape[:2]  # height 和 width 是通过 shape 获得

# 将图像分割为16行30列的网格
rows, cols = 16, 30

# 计算每个网格的宽度和高度
cell_width = width / cols  # 每个网格的宽度，整数除法
cell_height = height / rows  # 每个网格的高度，整数除法

# 排除的颜色范围 (BGR格式)
excluded_colors = np.array([[65, 58, 50], [104, 68, 56], [105, 87, 41]])
lower_bounds = excluded_colors - 10  # 下界
upper_bounds = excluded_colors + 10    # 上界

# 遍历每个网格单元并进行颜色检测
for row in range(num_rows):
    for col in range(num_cols):
        # 计算每个单元格的坐标，使用浮动边界
        x_start = int(col * cell_width)
        y_start = int(row * cell_height)
        x_end = int((col + 1) * cell_width)
        y_end = int((row + 1) * cell_height)

        # 截取每个单元格区域
        cell = hsv_image[y_start:y_end, x_start:x_end]

        # 计算平均颜色值
        avg_color = cv2.mean(cell)[:3]  # 提取平均颜色的 HSV 值

        # 初始化标记为隐藏 ('H')
        detected_value = 'H'

        # 根据平均颜色值决定矩阵的值
        for num, (lower, upper) in color_ranges.items():
            h_check = lower[0] <= avg_color[0] <= upper[0]
            s_check = lower[1] <= avg_color[1] <= upper[1]
            v_check = lower[2] <= avg_color[2] <= upper[2]
            
            if h_check and s_check and v_check:
                detected_value = num
                break

        # 将检测到的值和平均颜色填入矩阵
        matrix[row][col] = (detected_value, avg_color)  # 存储数字和颜色值

# 打印矩阵的维度
print(f"{num_cols}x{num_rows}x3")

# 打印矩阵内容，并显示每个网格的HSV均值
for row in matrix:
    print(" ".join([f"{cell[0]} ({int(cell[1][0])}, {int(cell[1][1])}, {int(cell[1][2])})" for cell in row]))

# 构建保存的字符串
output_str = f"{num_cols}x{num_rows}x99\n"
for row in matrix:
    output_str += "".join([f"{cell[0]}" for cell in row]) + "\n"

# 获取当前时间戳，用于命名文件
# timestamp = int(time.time())
# filename = f"{timestamp}.mine"
filename = f"map.mine"

# 保存为文件
with open(filename, "w") as file:
    file.write(output_str)

# 输出文件名
print(f"矩阵保存为文件: {filename}")

30x16x3
1 (103, 73, 89) H (104, 52, 78) 2 (89, 76, 93) 3 (128, 79, 100) H (104, 48, 81) H (104, 52, 77) 1 (103, 75, 88) 0 (104, 62, 61) 0 (104, 61, 62) 0 (104, 62, 61) 1 (103, 75, 88) H (104, 51, 78) H (104, 48, 81) H (104, 48, 81) H (104, 48, 81) H (104, 48, 81) H (104, 47, 83) H (104, 49, 80) H (104, 47, 83) H (104, 49, 80) H (104, 47, 83) H (104, 49, 81) H (104, 49, 81) H (104, 48, 81) H (104, 49, 81) H (104, 51, 78) 1 (104, 76, 87) 0 (104, 61, 62) 0 (104, 62, 62) 0 (104, 61, 62)
1 (103, 73, 91) 2 (88, 77, 91) H (104, 50, 79) 5 (71, 104, 100) H (104, 50, 79) 4 (116, 74, 99) 1 (103, 74, 90) 1 (103, 74, 87) 2 (87, 77, 93) 2 (89, 76, 90) 3 (130, 78, 104) 3 (129, 77, 102) H (104, 49, 80) H (104, 49, 80) H (104, 49, 80) H (104, 49, 80) H (104, 48, 81) H (104, 50, 78) H (104, 48, 81) H (104, 50, 78) H (104, 48, 81) H (104, 49, 80) H (104, 49, 80) H (104, 49, 80) H (104, 49, 80) 3 (129, 78, 102) 3 (129, 78, 104) 1 (103, 73, 88) 2 (88, 78, 93) 1 (103, 73, 88)
1 (103, 74, 89) 2 (88, 78, 90) 